In [1]:
import pandas as pd
import numpy as np
import collections
import re
import random
import operator
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression

In [2]:
# import training data and print for sizes and info
dfTrain = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")
print(dfTrain)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [3]:
dfTotal = pd.concat([dfTrain, dfTest], axis=0)
dfTotal.set_index("PassengerId", inplace=True)
dfTotal

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S


In [4]:
dfTotal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    float64
 1   Pclass    1309 non-null   int64  
 2   Name      1309 non-null   object 
 3   Sex       1309 non-null   object 
 4   Age       1046 non-null   float64
 5   SibSp     1309 non-null   int64  
 6   Parch     1309 non-null   int64  
 7   Ticket    1309 non-null   object 
 8   Fare      1308 non-null   float64
 9   Cabin     295 non-null    object 
 10  Embarked  1307 non-null   object 
dtypes: float64(3), int64(3), object(5)
memory usage: 122.7+ KB


In [5]:
dfTotal["Pclass"].value_counts()

3    709
1    323
2    277
Name: Pclass, dtype: int64

In [6]:
dfTotal["Sex"].value_counts()

male      843
female    466
Name: Sex, dtype: int64

In [7]:
dfTotal["SibSp"].value_counts()

0    891
1    319
2     42
4     22
3     20
8      9
5      6
Name: SibSp, dtype: int64

In [8]:
dfTotal["Parch"].value_counts()

0    1002
1     170
2     113
3       8
4       6
5       6
6       2
9       2
Name: Parch, dtype: int64

In [9]:
dfTotal["Ticket"].value_counts()

CA. 2343      11
CA 2144        8
1601           8
PC 17608       7
347077         7
              ..
C.A. 5547      1
323592         1
234604         1
C.A. 31026     1
330968         1
Name: Ticket, Length: 929, dtype: int64

In [10]:
dfTotal["Embarked"].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [11]:
dfTotal[dfTotal["Fare"].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1044,NaN,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [12]:
dfTotal[dfTotal["Fare"].isnull()] = dfTotal[dfTotal["Pclass"]==3].Fare.mean()

In [13]:
dfTotal[dfTotal["Embarked"].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1.0,1.0,"Icard, Miss. Amelie",female,38.0,0.0,0.0,113572,80.0,B28,NaN
830,1.0,1.0,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0.0,0.0,113572,80.0,B28,NaN


In [14]:
dfTotal["Embarked"].value_counts()

S                     913
C                     270
Q                     123
13.302888700564973      1
Name: Embarked, dtype: int64

In [15]:
dfTotal[dfTotal["Embarked"]=="Q"].Survived.value_counts()

0.0    47
1.0    30
Name: Survived, dtype: int64

In [16]:
dfTotal[dfTotal["Embarked"]=="C"].Survived.value_counts()

1.0    93
0.0    75
Name: Survived, dtype: int64

In [17]:
dfTotal[dfTotal["Embarked"]=="S"].Survived.value_counts()

0.0    427
1.0    217
Name: Survived, dtype: int64

In [18]:
dfTotal["Embarked"].replace({"S":1, "Q":2, "C":3}, inplace=True)

In [19]:
dfTotal[dfTotal["Embarked"].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1.0,1.0,"Icard, Miss. Amelie",female,38.0,0.0,0.0,113572,80.0,B28,NaN
830,1.0,1.0,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0.0,0.0,113572,80.0,B28,NaN


In [20]:
dfTotal["Sex"].replace({"female":0, "male":1}, inplace=True)

In [21]:
Xtrain = dfTotal[dfTotal["Embarked"].notnull()]

In [22]:
Xtrain

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3.0,"Braund, Mr. Owen Harris",1.0,22.0,1.0,0.0,A/5 21171,7.2500,NaN,1.0
2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.0,38.0,1.0,0.0,PC 17599,71.2833,C85,3.0
3,1.0,3.0,"Heikkinen, Miss. Laina",0.0,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,1.0
4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.0,35.0,1.0,0.0,113803,53.1000,C123,1.0
5,0.0,3.0,"Allen, Mr. William Henry",1.0,35.0,0.0,0.0,373450,8.0500,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3.0,"Spector, Mr. Woolf",1.0,NaN,0.0,0.0,A.5. 3236,8.0500,NaN,1.0
1306,NaN,1.0,"Oliva y Ocana, Dona. Fermina",0.0,39.0,0.0,0.0,PC 17758,108.9000,C105,3.0
1307,NaN,3.0,"Saether, Mr. Simon Sivertsen",1.0,38.5,0.0,0.0,SOTON/O.Q. 3101262,7.2500,NaN,1.0


In [23]:
usableCols = ["Pclass", "Sex", "SibSp", "Parch", "Fare"]
ytrain = Xtrain["Embarked"]
Xtrain = Xtrain[usableCols]

Xtrain.head()
ytrain.head()

PassengerId
1    1.0
2    3.0
3    1.0
4    1.0
5    1.0
Name: Embarked, dtype: float64

In [24]:
Xtrain = Xtrain.astype("float64")

In [25]:
ytrain = ytrain.astype("int64")

In [26]:
clf = MLPClassifier(hidden_layer_sizes=(20,20,20))
clf.fit(Xtrain.to_numpy(), ytrain.to_numpy())

C:\Users\niall\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(20, 20, 20))

In [27]:
Xtest = dfTotal[dfTotal["Embarked"].isnull()]

In [28]:
Xtest = Xtest[usableCols]

In [29]:
clf.predict(Xtest)

array([3, 3], dtype=int64)

In [30]:
dfTotal["Embarked"].fillna(3, inplace=True)

In [31]:
dfTotal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  892 non-null    float64
 1   Pclass    1309 non-null   float64
 2   Name      1309 non-null   object 
 3   Sex       1309 non-null   float64
 4   Age       1046 non-null   float64
 5   SibSp     1309 non-null   float64
 6   Parch     1309 non-null   float64
 7   Ticket    1309 non-null   object 
 8   Fare      1309 non-null   float64
 9   Cabin     296 non-null    object 
 10  Embarked  1309 non-null   float64
dtypes: float64(8), object(3)
memory usage: 122.7+ KB


In [32]:
dfTotal.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
Survived,1.000000,0.036487,0.137280,-0.084617,0.215936,0.368174,0.183775,0.413216
Pclass,0.036487,1.000000,0.295681,-0.393828,0.164270,0.146378,-0.528675,-0.039643
Sex,0.137280,0.295681,1.000000,0.025907,0.106169,0.066032,-0.156092,0.142019
Age,-0.084617,-0.393828,0.025907,1.000000,-0.237057,-0.150895,0.179073,0.065405
SibSp,0.215936,0.164270,0.106169,-0.237057,1.000000,0.449588,0.148276,0.060025
Parch,0.368174,0.146378,0.066032,-0.150895,0.449588,1.000000,0.200713,0.101495
Fare,0.183775,-0.528675,-0.156092,0.179073,0.148276,0.200713,1.000000,0.219691
Embarked,0.413216,-0.039643,0.142019,0.065405,0.060025,0.101495,0.219691,1.000000


In [33]:
dfTotal

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3.0,"Braund, Mr. Owen Harris",1.0,22.0,1.0,0.0,A/5 21171,7.2500,NaN,1.0
2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.0,38.0,1.0,0.0,PC 17599,71.2833,C85,3.0
3,1.0,3.0,"Heikkinen, Miss. Laina",0.0,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,1.0
4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.0,35.0,1.0,0.0,113803,53.1000,C123,1.0
5,0.0,3.0,"Allen, Mr. William Henry",1.0,35.0,0.0,0.0,373450,8.0500,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1305,NaN,3.0,"Spector, Mr. Woolf",1.0,NaN,0.0,0.0,A.5. 3236,8.0500,NaN,1.0
1306,NaN,1.0,"Oliva y Ocana, Dona. Fermina",0.0,39.0,0.0,0.0,PC 17758,108.9000,C105,3.0
1307,NaN,3.0,"Saether, Mr. Simon Sivertsen",1.0,38.5,0.0,0.0,SOTON/O.Q. 3101262,7.2500,NaN,1.0


## Now set ages using some regession technique

In [34]:
Xtrain = dfTotal[dfTotal["Age"].notna()]
Xtrain

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3.0,"Braund, Mr. Owen Harris",1.0,22.0,1.0,0.0,A/5 21171,7.2500,NaN,1.0
2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.0,38.0,1.0,0.0,PC 17599,71.2833,C85,3.0
3,1.0,3.0,"Heikkinen, Miss. Laina",0.0,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,1.0
4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.0,35.0,1.0,0.0,113803,53.1000,C123,1.0
5,0.0,3.0,"Allen, Mr. William Henry",1.0,35.0,0.0,0.0,373450,8.0500,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1301,NaN,3.0,"Peacock, Miss. Treasteall",0.0,3.0,1.0,1.0,SOTON/O.Q. 3101315,13.7750,NaN,1.0
1303,NaN,1.0,"Minahan, Mrs. William Edward (Lillian E Thorpe)",0.0,37.0,1.0,0.0,19928,90.0000,C78,2.0
1304,NaN,3.0,"Henriksson, Miss. Jenny Lovisa",0.0,28.0,0.0,0.0,347086,7.7750,NaN,1.0


In [35]:
usableColumns = ["Pclass", "Sex", "SibSp", "Parch", "Fare", "Embarked"]
ytrain = Xtrain["Age"].astype("int16")
Xtrain = Xtrain[usableCols]

In [36]:
clf = LinearRegression()
clf.fit(Xtrain, ytrain)

LinearRegression()

In [37]:
Xtest = dfTotal[dfTotal["Age"].isnull()]
Xtest = Xtest[usableCols]
missingAge = clf.predict(Xtest)
hasAge = dfTotal[dfTotal["Age"].notnull()]
hasntAge = dfTotal[dfTotal["Age"].isnull()]
hasntAge["Age"] = missingAge
dfTotal = pd.concat([hasAge, hasntAge], axis=0)
dfTotal

<ipython-input-37-9fb899978b21>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hasntAge["Age"] = missingAge


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0.0,3.0,"Braund, Mr. Owen Harris",1.0,22.000000,1.0,0.0,A/5 21171,7.2500,NaN,1.0
2,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0.0,38.000000,1.0,0.0,PC 17599,71.2833,C85,3.0
3,1.0,3.0,"Heikkinen, Miss. Laina",0.0,26.000000,0.0,0.0,STON/O2. 3101282,7.9250,NaN,1.0
4,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0.0,35.000000,1.0,0.0,113803,53.1000,C123,1.0
5,0.0,3.0,"Allen, Mr. William Henry",1.0,35.000000,0.0,0.0,373450,8.0500,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1300,NaN,3.0,"Riordan, Miss. Johanna Hannah""""",0.0,22.955606,0.0,0.0,334915,7.7208,NaN,2.0
1302,NaN,3.0,"Naughton, Miss. Hannah",0.0,22.955733,0.0,0.0,365237,7.7500,NaN,2.0
1305,NaN,3.0,"Spector, Mr. Woolf",1.0,27.742003,0.0,0.0,A.5. 3236,8.0500,NaN,1.0


In [38]:
dfTotal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 1 to 1309
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  892 non-null    float64
 1   Pclass    1309 non-null   float64
 2   Name      1309 non-null   object 
 3   Sex       1309 non-null   float64
 4   Age       1309 non-null   float64
 5   SibSp     1309 non-null   float64
 6   Parch     1309 non-null   float64
 7   Ticket    1309 non-null   object 
 8   Fare      1309 non-null   float64
 9   Cabin     296 non-null    object 
 10  Embarked  1309 non-null   float64
dtypes: float64(8), object(3)
memory usage: 122.7+ KB


In [39]:
dfTotal.sort_index(inplace=True)

In [41]:
usableCols=["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
Xtrain = dfTotal.loc[dfTotal["Survived"].notnull(), usableCols]
ytrain = dfTotal.loc[dfTotal["Survived"].notnull(), "Survived"].astype("int16")
Xtest = dfTotal.loc[dfTotal["Survived"].isnull(), usableCols]
ytest = dfTotal.loc[dfTotal["Survived"].isnull(), "Survived"]
clf = MLPClassifier(hidden_layer_sizes=(50,50,50))
clf.fit(Xtrain, ytrain)

MLPClassifier(hidden_layer_sizes=(50, 50, 50))

In [45]:
ytest["Survived"] = clf.predict(Xtest)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [44]:
ytest

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,